# NB04: ABMC並列性能評価

## 目的

ABMC (Algebraic Block Multi-Color) 順序付けによるICCGの並列性能を評価する:

1. **スレッドスケーリング**: 1, 2, 4, 8スレッドでの性能変化
2. **Setup vs Solve 分離計測**: IC分解(Setup)と反復法(Solve)の時間内訳
3. **ABMC vs Level-Schedule vs BDDC**: 異なる並列化手法の壁時計時間比較

## 背景

### IC前処理の並列化の課題

IC(0)前処理の前進・後退代入は本質的に逐次的であり、単純には並列化できない。
ABMC順序付けはこのボトルネックを解消する:

| 手法 | Setup (IC分解) | Apply (前進・後退代入) |
|------|---------------|-------------------|
| Level-Schedule | 逐次 | レベルごとに parallel_for |
| ABMC | 色ごとに parallel_for | 色ごとに parallel_for |

### ABMCの構造

```
色0: [Block0, Block1, ...] ← parallel_for
色1: [Block5, Block6, ...] ← parallel_for
  :      :                       :
```

- 色は逐次処理（色間依存）
- 同色内のブロックは並列処理（独立）
- ブロック内の行は逐次処理

### ABMCの2つの効果

1. **並列性**: ブロック単位の粗粒度並列化で、Level-Scheduleより効率的なスレッド利用
2. **キャッシュ局所性**: BFSによるブロック化で近接行がまとまり、シングルスレッドでも高速

In [ ]:
import ngsolve
from ngsolve import *
from netgen.occ import *
import time
from math import pi

from sparsesolv_ngsolve import SparseSolvSolver, BDDCPreconditioner
from ngsolve.krylovspace import CGSolver
print("Setup complete")

## 1. テスト問題: トーラスコイル (HCurl curl-curl)

NB02と同じトーラスコイル問題。HCurl order=2, 約148K DOFs。

In [ ]:
p1 = Pnt(0.6, 0, -0.1)
p2 = Pnt(0.4, 0, -0.1)
p3 = Pnt(0.4, 0,  0.1)
p4 = Pnt(0.6, 0,  0.1)
rect = Wire([Segment(p1, p2), Segment(p2, p3), Segment(p3, p4), Segment(p4, p1)])
coil_shape = Revolve(rect, Axis(Pnt(0,0,0), Vec(0,0,1)), 360)
coil_shape.maxh = 0.05

sphere = Sphere(Pnt(0,0,0), 1.0).bc("outer")
shape = Glue([sphere, coil_shape])
shape.solids[0].mat("air")
shape.solids[1].mat("coil")

mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.1))
mesh.Curve(3)
print(f"メッシュ: {mesh.ne} 要素")

eps = 1e-6
fes = HCurl(mesh, order=2, dirichlet="outer", nograds=True)
u, v = fes.TrialFunction(), fes.TestFunction()
a = BilinearForm(curl(u)*curl(v)*dx + eps*u*v*dx)
a.Assemble()

J = CoefficientFunction((1/(0.2*0.2)*y/sqrt(x**2+y**2),
                          -1/(0.2*0.2)*x/sqrt(x**2+y**2), 0))
f = LinearForm(J*v*dx("coil"))
f.Assemble()
mat = a.mat
freedofs = fes.FreeDofs()
print(f"自由度数: {fes.ndof}")

## 2. スレッドスケーリング

ABMC有効ICCGとLevel-Schedule ICCGをスレッド数を変えて比較する。
Setup時間とSolve時間を分離計測する。

In [ ]:
nruns = 3
thread_counts = [1, 2, 4, 8]

results = []

for nt in thread_counts:
    SetNumThreads(nt)
    print(f"\n--- {nt} threads ---")

    for method_name, use_abmc in [("ABMC", True), ("Level-Sched", False)]:
        # Warmup
        with TaskManager():
            _ = SparseSolvSolver(mat, "ICCG", freedofs, use_abmc=use_abmc,
                                 tol=1e-8, shift=1.5)

        setup_times, solve_times, iters = [], [], 0
        for run in range(nruns):
            with TaskManager():
                t0 = time.perf_counter()
                solver = SparseSolvSolver(mat, "ICCG", freedofs,
                                         use_abmc=use_abmc, abmc_num_colors=8,
                                         tol=1e-8, maxiter=2000, shift=1.5)
                solver.diagonal_scaling = True
                t_setup = time.perf_counter() - t0

                gfu = GridFunction(fes)
                t0 = time.perf_counter()
                res = solver.Solve(f.vec, gfu.vec)
                t_solve = time.perf_counter() - t0
            setup_times.append(t_setup)
            solve_times.append(t_solve)
            iters = res.iterations

        avg_setup = sum(setup_times) / nruns
        avg_solve = sum(solve_times) / nruns
        results.append((nt, method_name, avg_setup * 1000, avg_solve * 1000, iters))
        label = "ABMC" if use_abmc else "LvSch"
        print(f"  {label}: setup={avg_setup*1000:.1f}ms, solve={avg_solve*1000:.1f}ms, "
              f"total={(avg_setup+avg_solve)*1000:.1f}ms, iters={iters}")

In [ ]:
# 結果表
EQ, DA = chr(61), chr(45)
print(f"\nトーラスコイル HCurl order=2, {fes.ndof} DOFs")
print(f"{EQ*80}")
print(f"{'Threads':>7} | {'Method':>11} | {'Setup(ms)':>9} | {'Solve(ms)':>9} | "
      f"{'Total(ms)':>9} | {'Iters':>5} | {'ms/iter':>7}")
print(f"{DA*80}")
for nt, method, s, v, it in results:
    ms_per = v / it if it > 0 else 0
    print(f"{nt:>7} | {method:>11} | {s:>9.1f} | {v:>9.1f} | {s+v:>9.1f} | {it:>5} | {ms_per:>7.1f}")
print(f"{EQ*80}")

# スケーリング (Solve時間のみ)
print(f"\n--- Solveスケーリング (1スレッド基準) ---")
for method_name in ["ABMC", "Level-Sched"]:
    base = [v for nt, m, _, v, _ in results if m == method_name and nt == 1]
    if not base:
        continue
    bt = base[0]
    for nt, m, _, v, _ in results:
        if m == method_name:
            speedup = bt / v if v > 0 else 0
            print(f"  {method_name:>11} {nt}T: {speedup:.2f}x")

### 観察

- **1スレッドでもABMCが高速**: ABMCのBFSブロック化によるキャッシュ局所性改善の効果
- **ms/iter**: ABMCが全スレッド数でLevel-Scheduleより高速。ブロック単位の粗粒度並列化が効果的
- **反復数の違い**: ABMCは行列をリオーダリングするためIC分解の結果が変わり、反復数が異なる
- **Setup時間**: 全ケースで1-2msで無視できるレベル。性能差はSolve（反復ごとのApply）で決まる

## 3. ABMC vs BDDC (8スレッド)

ABMCはSetupが軽量、BDDCは反復数が少ない。
問題規模によって最適な手法が異なる。

In [ ]:
SetNumThreads(8)

comparison = []

# ABMC ICCG (8 threads)
setup_times, solve_times = [], []
for run in range(nruns):
    with TaskManager():
        t0 = time.perf_counter()
        solver = SparseSolvSolver(mat, "ICCG", freedofs,
                                 use_abmc=True, abmc_num_colors=8,
                                 tol=1e-8, maxiter=2000, shift=1.5)
        solver.diagonal_scaling = True
        t_setup = time.perf_counter() - t0

        gfu = GridFunction(fes)
        t0 = time.perf_counter()
        res = solver.Solve(f.vec, gfu.vec)
        t_solve = time.perf_counter() - t0
    setup_times.append(t_setup)
    solve_times.append(t_solve)

avg_setup = sum(setup_times) / nruns
avg_solve = sum(solve_times) / nruns
comparison.append(("ICCG+ABMC", res.iterations, avg_setup, avg_solve))
print(f"ICCG+ABMC: {res.iterations} iters, setup={avg_setup:.3f}s, "
      f"solve={avg_solve:.3f}s, total={avg_setup+avg_solve:.3f}s")

# BDDC (8 threads)
setup_times, solve_times = [], []
for run in range(nruns):
    with TaskManager():
        t0 = time.perf_counter()
        bddc = BDDCPreconditioner(a, fes, coarse_inverse="sparsecholesky")
        t_setup = time.perf_counter() - t0

        gfu = GridFunction(fes)
        t0 = time.perf_counter()
        inv = CGSolver(mat=mat, pre=bddc, maxiter=500, tol=1e-8, printrates=False)
        gfu.vec.data = inv * f.vec
        t_solve = time.perf_counter() - t0
    setup_times.append(t_setup)
    solve_times.append(t_solve)

avg_setup = sum(setup_times) / nruns
avg_solve = sum(solve_times) / nruns
comparison.append(("BDDC", inv.iterations, avg_setup, avg_solve))
print(f"BDDC:      {inv.iterations} iters, setup={avg_setup:.3f}s, "
      f"solve={avg_solve:.3f}s, total={avg_setup+avg_solve:.3f}s")

In [ ]:
EQ, DA = chr(61), chr(45)
print(f"\nトーラスコイル HCurl order=2, {fes.ndof} DOFs, 8 threads")
print(f"{EQ*70}")
print(f"{'Method':>12} | {'Iters':>5} | {'Setup(s)':>8} | {'Solve(s)':>8} | "
      f"{'Total(s)':>8} | {'Setup%':>6}")
print(f"{DA*70}")
for name, iters, s, v in comparison:
    pct = s / (s + v) * 100
    print(f"{name:>12} | {iters:>5} | {s:>8.3f} | {v:>8.3f} | {s+v:>8.3f} | {pct:>5.1f}%")
print(f"{EQ*70}")

print(f"\nBDDC Setupコスト: {comparison[1][2]:.3f}s")
print(f"ABMC Setupコスト: {comparison[0][2]*1000:.1f}ms")
print(f"Setupコスト比: BDDC / ABMC = {comparison[1][2]/comparison[0][2]:.0f}x")

## まとめ

### 性能特性

| 手法 | Setupコスト | 反復数 | ms/iter (8T) | 並列スケーリング |
|------|------------|--------|------------|----------------|
| Level-Schedule | ~2ms | 少 | ~7.5 | 低（レベルが細かい） |
| ABMC | ~2ms | やや多 | ~5.6 | 高（ブロック単位） |
| BDDC | ~1100ms | 少 | --- | 高（素子単位） |

### ABMCの効果

1. **反復ごとのApplyが高速**: ブロック単位の粗粒度並列化により、8TでLevel-Scheduleの~25%高速
2. **キャッシュ局所性**: BFSブロック化で近接行がまとまり、1スレッドでも高速
3. **Setupコスト**: IC分解のみで無視できるレベル。BDDCのSetupの~500倍高速

### 適用指針

- **ABMC ICCG**: BDDCのセットアップコストが割に合わない中規模問題で有効
- **BDDC**: 大規模問題では反復数の少なさが圧倒的に有利
- **Level-Schedule**: シングルスレッド環境ではABMCのオーバーヘッドがない分有利な場合もある